####OGC Element vs Token counts

Unlike the other metadata token/element evaluation, we're going to be fairly explicit with the expected text locations here. 

Does the service description section contain text? Does a "normal" layer contain text?

Totals for the layers are not very meaningful - these are automatically generated responses so we can expect that the layers conform to the same structure throughout a GetCapabilities response. Internally to each response and most likely across the set of GetCapabilities per service and server.

In [1]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import *
from sqlalchemy import and_
from semproc.xml_utils import *
from semproc.parser import Parser
from mpp.models import Response
from datetime import datetime
# from lxml import etree

In [2]:
# load the postgres connection file
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [3]:
# the token sets
# assumes executing the token paths from the 
# element described by the key

paths = {
    "Service": [
        ['Name'],
        ['Title'],
        ['Abstract'],
        ['KeywordList', 'Keyword'],
        ['OnlineResource', '@href'],
        ['ContactInformation', 'ContactPersonPrimary', 'ContactPerson'],
        ['ContactInformation', 'ContactPersonPrimary', 'ContactOrganization'],
        ['ContactInformation', 'ContactPosition'],
        ['ContactInformation', 'ContactAddress', 'AddressType'],
        ['ContactInformation', 'ContactAddress', 'Address'],
        ['ContactInformation', 'ContactAddress', 'City'],
        ['ContactInformation', 'ContactAddress', 'StateOrProvince'],
        ['ContactInformation', 'ContactAddress', 'PostCode'],
        ['ContactInformation', 'ContactAddress', 'Country'],
        ['ContactInformation', 'ContactVoiceTelephone'],
        ['ContactInformation', 'ContactFacsimileTelephone'],
        ['ContactInformation', 'ContactElectronicMailAddress'],
        ['Fees'],
        ['AccessConstraints']
    ],
    "ServiceIdentification": [
        ['Title'],
        ['Abstract'],
        ['Keywords', 'Keyword'],
        ['ServiceType'],
        ['ServiceTypeVersion'],
        ['Fees'],
        ['AccessConstraints']
    ],
    # need to run service provider for the ows versions
    'ServiceProvider': [
        ['ProviderName'],
        ['ServiceContact', 'IndividualName'],
        ['ServiceContact', 'PositionName'],
        ['ServiceContact', 'ContactInfo', 'Phone', 'Voice'],
        ['ServiceContact', 'ContactInfo', 'Phone', 'Facsimile'],
        ['ServiceContact', 'ContactInfo', 'Address', 'DeliveryPoint'],
        ['ServiceContact', 'ContactInfo', 'Address', 'City'],
        ['ServiceContact', 'ContactInfo', 'Address', 'AdministrativeArea'],
        ['ServiceContact', 'ContactInfo', 'Address', 'PostalCode'],
        ['ServiceContact', 'ContactInfo', 'Address', 'Country'],
        ['ServiceContact', 'ContactInfo', 'Address', 'ElectronicMailAddress'],
        ['ServiceContact', 'ContactInfo', 'OnlineResource', '@href'],
        ['ServiceContact', 'ContactInfo', 'HoursOfService'],
        ['ServiceContact', 'ContactInfo', 'ContactInstructions'],
        ['Role']
    ]
}

layer_paths = {
    'Layer': [
        ['Title'],
        ['Name'],
        ['Abstract'],
        ['KeywordList', 'Keyword'],
        ['Identifier']
    ],
    'FeatureType': [
        ['Name'],
        ['Title'],
        ['Abstract'],
        ['Keywords', 'Keyword'],
        ['Identifier']
    ],
    'CoverageSummary': [
        ['Title'],
        ['Abstract'],
        ['Keywords', 'Keyword'],
        ['Identifier']
    ]
}

In [16]:
sql = """
with i as (
	select d.response_id, (e.value->'protocol')::text as ident
	from identities d, jsonb_array_elements(d.identity::jsonb) e
	where d.identity is not null 
		and e.value->>'protocol' = 'OGC'
)
select r.id, r.source_url_sha, r.source_url, r.cleaned_content
from responses r join i on i.response_id = r.id
;
"""

In [17]:
def _extract_tag(t):
    if not t:
        return
    return t.split('}')[-1]

def _taggify(e, x):
    x.reverse()
    tags = x + [e.tag] + [m.tag for m in e.iterancestors()]
    tags.reverse()

    try:
        return '/'.join([_extract_tag(t) for t in tags])
    except:
        return ''

def strip_text(elem, arr):
    for a in arr:
        if '@' in a[-1]:
            texts = [extract_attrib(elem, a)]
        else:
            texts = extract_items(elem, a)
        
        if not texts:
            continue
        
        yield _taggify(elem, a), texts

In [18]:
x = ''
for i, sha, url, content in session.execute(sql):
    try:
        parser = Parser(content.encode('utf-8'))
    except:
        continue
        
    x = parser.xml
    
    blobs = []
    for child in parser.xml:
        key = _extract_tag(child.tag)
        if not key in paths:
            continue
            
        arr = paths.get(key, [])
        elem = extract_elem(parser.xml, [key])
        
        extracted = [(tag, texts) for tag, texts in strip_text(elem, arr)]
        if extracted:
            blobs.append({
                "num_possible": len(arr),
                "extracted": extracted
            })
    
    if blobs:
        data = {
            "url": url,
            "blobs": blobs
        }
        with open('outputs/ogc/%s.json' % i, 'w') as g:
            g.write(js.dumps(data, indent=4))
